In [33]:
import pandas as pd
import pymongo
from sqlalchemy import create_engine
import numpy as np
from config import pword

In [16]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [17]:
def randomIndex(low=0,high=10,size=10):
    index_list =[]
    while len(index_list) != size:
        num=np.random.randint(low, high,1)
        if num not in index_list:
            index_list.append(num[0])
    return sorted(list((index_list)))
        
def col_to_list(col):
    return list(col.find()), len(list(col.find()))

In [18]:
quotes_mdb=client.quotes_db
author_info_col=quotes_mdb.author_information_collection


author_list,len_author_list=col_to_list(author_info_col)

author_info_df=pd.DataFrame(author_list,index=randomIndex(100,300,len_author_list))


author_info_table = author_info_df[['name','born','description']].copy()
author_info_table.index.name='author_id'
author_info_table=author_info_table.drop_duplicates(subset=['name'])
author_info_table #sql table 1

,name,born,description
author_id,,,
102,Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
104,J.K. Rowling,"July 31, 1965 in Yate, South Gloucestershire, ...",\n See also: Robert GalbraithAlthough s...
108,Jane Austen,"December 16, 1775 in Steventon Rectory, Hampsh...",\n Jane Austen was an English novelist ...
109,Marilyn Monroe,"June 01, 1926 in The United States",\n Marilyn Monroe (born Norma Jeane Mor...
113,André Gide,"November 22, 1869 in Paris, France",\n André Paul Guillaume Gide was a Fren...
116,Thomas A. Edison,"February 11, 1847 in Milan, Ohio, The United S...",\n Thomas Alva Edison was an American i...
117,Eleanor Roosevelt,"October 11, 1884 in The United States",\n Anna Eleanor Roosevelt was an Americ...
118,Steve Martin,"August 14, 1945 in Waco, Texas, The United States","\n Stephen Glenn ""Steve"" Martin is an A..."
123,Bob Marley,"February 06, 1945 in Nine Mile, Saint Ann, Jam...","\n Robert ""Bob"" Nesta Marley OM was a J..."


In [19]:
tags_col=quotes_mdb.tags_collection
tags_list,len_tags_list=col_to_list(tags_col)
tags_df=pd.DataFrame(tags_list,index=randomIndex(301,500,len_tags_list))

tags_table = tags_df[['tag']].copy()
tags_table=tags_table.rename(columns={'tag':'tag_name'})
tags_table.index.name = 'tag_id'
tags_table #sql table 2

,tag_name
tag_id,
301,change
302,deep-thoughts
304,thinking
305,world
306,abilities
...,...
489,difficult
490,grown-ups
491,write


#### 

In [20]:
quotes_col=quotes_mdb.tag_relation_collection
quotes_list,len_quotes_list=col_to_list(quotes_col)
quotes_df= pd.DataFrame(quotes_list,index=randomIndex(501,700,len_quotes_list))
# quotes_df

In [21]:
quotes_and_tag_table = quotes_df[["quote_text",'tag']].copy()
quotes_and_tag_table=quotes_and_tag_table.rename(columns={'tag':'tag_list'})
quotes_and_tag_table.index.name = 'quote_id'
# quotes_and_tag_table

In [22]:
quoteid_tag=[]

for index, row in quotes_and_tag_table.iterrows():
    for tag in row['tag_list']:
        quoteid_tag.append((index,tag))
quote_tag_df=pd.DataFrame(quoteid_tag,columns=['quote_id','tag_name'])


In [9]:
# quote_tag_df

In [23]:
quotes_and_tag_relations_table=tags_table.reset_index().merge(quote_tag_df,on='tag_name')

In [24]:
quotes_and_tag_relations_table=quotes_and_tag_relations_table[['tag_id','quote_id']].copy()

In [25]:
quotes_and_tag_relations_table=quotes_and_tag_relations_table.set_index('tag_id') #sql table 3

In [26]:
quotes_and_tag_relations_table

,quote_id
tag_id,
301,504
302,504
304,504
304,640
305,504
...,...
489,687
490,687
491,687


In [27]:
quotes_table=quotes_and_tag_table[['quote_text']].copy()
# quotes_table

In [28]:
quotes_col=quotes_mdb.quotes_collection
quotes_and_author_df=pd.DataFrame(list(quotes_col.find()))
quotes_and_author_table=quotes_and_author_df[['quote_text','author_name']].copy().rename(columns={'author_name':'name'})

In [29]:
quotes_and_author_table=quotes_and_author_table.merge(author_info_table.reset_index(), on='name')
# quotes_and_author_table

In [30]:
quotes_and_author_table=(quotes_and_author_table[['quote_text','author_id']].copy()).merge(quotes_and_tag_table.reset_index(),on="quote_text").drop(['tag_list'],axis=1)

In [31]:
quotes_and_author_table=quotes_and_author_table.set_index('quote_id')

In [32]:
quotes_and_author_table

,quote_text,author_id
quote_id,,
504,“The world as we have created it is a process ...,102
507,“There are only two ways to live your life. On...,102
513,“Try not to become a man of success. Rather be...,102
529,"“If you can't explain it to a six year old, yo...",102
560,"“If you want your children to be intelligent, ...",102
...,...,...
683,“To die will be an awfully big adventure.”,286
684,“It takes courage to grow up and become who yo...,287
685,“But better to get hurt by the truth than comf...,291


In [34]:

                
rds_connection_string = f"postgres:{pword}@localhost:5432/Quotes_DB"#<=== username is postgres and mypassword is next
engine = create_engine(f'postgresql://{rds_connection_string}')

# Confirm tables
engine.table_names()

['quote_tag_relation', 'tags', 'author_info', 'quotes']

In [27]:


author_info_table.to_sql(name='author_info',con=engine,if_exists='append',index=True)

quotes_and_author_table.to_sql(name='quotes',con=engine,if_exists='append',index=True)

tags_table.to_sql(name='tags',con=engine,if_exists='append',index=True)

quotes_and_tag_relations_table.to_sql(name='quote_tag_relation',con=engine,if_exists='append',index=True)